In [33]:
# Dependencies
from bs4 import BeautifulSoup
from splinter import Browser
import requests, json
import pymongo
import pandas as pd

In [34]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [35]:
# Define database and collection
db = client.brazilian_fires
collection = db.items

In [36]:
#import CSV
fire_data = pd.read_csv("C:/Users/Nathan Deering/Documents/DAB/GL/UNCC/01-Lesson-Plans/19-Project-2/fire_archive_M6_80460.csv")
fire_data_df = pd.DataFrame(fire_data)
fire_data_df

#remove fire types 1, 2, 3. Fire type 0 is vegetation fire
fire_data_df = fire_data_df[fire_data_df.type != 1]
fire_data_df = fire_data_df[fire_data_df.type != 2]
fire_data_df = fire_data_df[fire_data_df.type != 3]
fire_data_df = fire_data_df[fire_data_df.confidence >= 95]
fire_data_df = fire_data_df[(fire_data_df['acq_date'] > '2007-12-31')].reset_index()
nasa_fire_data_df = fire_data_df[['acq_date','latitude', 'longitude', 'brightness','confidence','bright_t31','frp']].copy()
nasa_fire_data_df



,acq_date,latitude,longitude,brightness,confidence,bright_t31,frp
0,2008-01-01,-9.2096,-36.8779,360.2,100,314.7,92.0
1,2008-01-01,-6.0107,-38.3172,355.1,97,311.2,81.9
2,2008-01-01,-6.0090,-38.3049,362.1,100,313.4,109.5
3,2008-01-02,-20.2190,-41.8678,325.5,100,291.0,30.3
4,2008-01-02,-9.7480,-36.4379,333.9,100,292.5,78.5
5,2008-01-02,-9.7455,-36.4220,342.0,100,293.0,108.4
6,2008-01-02,-9.9117,-35.9671,326.4,100,293.9,59.2
7,2008-01-02,-9.9221,-36.5258,317.6,95,286.6,36.7
8,2008-01-02,-16.1225,-43.5139,354.1,97,308.4,57.8
9,2008-01-02,-14.4657,-44.3452,376.0,100,291.1,132.8


In [37]:
def df_to_geojson(nasa_fire_data_df, properties, lat='latitude', lon='longitude'):
    geojson = {'type':'FeatureCollection', 'features':[]}
    for _, row in nasa_fire_data_df.iterrows():
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point',
                               'coordinates':[]}}
        feature['geometry']['coordinates'] = [row[lon],row[lat]]
        for prop in properties:
            feature['properties'][prop] = row[prop]
        geojson['features'].append(feature)
    return geojson

In [38]:
useful_columns = ['acq_date', 'latitude', 'longitude', 'brightness','confidence','bright_t31','frp']
geojson_dict = df_to_geojson(nasa_fire_data_df, properties=useful_columns)
geojson_str = json.dumps(geojson_dict, indent=2)

In [39]:
# save the geojson result to a file
output_filename = 'brazilian_fires.json'
with open(output_filename, 'w') as output_file:
    output_file.write('var dataset = {};'.format(geojson_str))
    
# how many features did we save to the geojson file?
print('{} geotagged features saved to file'.format(len(geojson_dict['features'])))

431934 geotagged features saved to file


In [40]:
data = nasa_fire_data_df.to_dict(orient='records') 

collection.insert_many(data)

#collection.insert_many(geojson_dict)